<a href="https://colab.research.google.com/github/hasan-sh/advanced-nlp/blob/main/DependencyFeatures_gabhoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import the package
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7ee653976e30cdd7c606b17ebb851f9d30ae6ddc94233a6ba23ac688aa236996
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [2]:
# Download an English model into the default directory
print("Downloading English model...")
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [3]:
# Build an English pipeline, with all processors by default
print("Building an English pipeline...")
en_nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Building an English pipeline...


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [4]:
# Processing English text
en_doc = en_nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
print(type(en_doc))

<class 'stanza.models.common.doc.Document'>


##GABRIEL's SANDBOX - unecessary for assignment

In [5]:
for i, sent in enumerate(en_doc.sentences):
    print("[Sentence {}]".format(i+1))
    for word in sent.words:
        print("{:12s}\t{:12d}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.id, word.lemma, word.pos, word.head, word.deprel))
    print("")

[Sentence 1]
Barack      	           1	Barack      	PROPN 	4	nsubj:pass  
Obama       	           2	Obama       	PROPN 	1	flat        
was         	           3	be          	AUX   	4	aux:pass    
born        	           4	bear        	VERB  	0	root        
in          	           5	in          	ADP   	6	case        
Hawaii      	           6	Hawaii      	PROPN 	4	obl         
.           	           7	.           	PUNCT 	4	punct       

[Sentence 2]
He          	           1	he          	PRON  	3	nsubj:pass  
was         	           2	be          	AUX   	3	aux:pass    
elected     	           3	elect       	VERB  	0	root        
president   	           4	president   	NOUN  	3	xcomp       
in          	           5	in          	ADP   	6	case        
2008        	           6	2008        	NUM   	3	obl         
.           	           7	.           	PUNCT 	3	punct       



In [6]:
print("Mention text\tType\tStart-End")
for ent in en_doc.ents:
    print("{}\t{}\t{}-{}".format(ent.text, ent.type, ent.start_char, ent.end_char))

Mention text	Type	Start-End


In [7]:

#en_doc.sentences[1].render_tree()

In [8]:
for token in en_doc.sentences[1].tokens:
  children = [child.text for child in token.children] if hasattr(token, 'children') else []
  print(token,children,"------------\n")


[
  {
    "id": 1,
    "text": "He",
    "lemma": "he",
    "upos": "PRON",
    "xpos": "PRP",
    "feats": "Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs",
    "head": 3,
    "deprel": "nsubj:pass",
    "start_char": 34,
    "end_char": 36
  }
] [] ------------

[
  {
    "id": 2,
    "text": "was",
    "lemma": "be",
    "upos": "AUX",
    "xpos": "VBD",
    "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
    "head": 3,
    "deprel": "aux:pass",
    "start_char": 37,
    "end_char": 40
  }
] [] ------------

[
  {
    "id": 3,
    "text": "elected",
    "lemma": "elect",
    "upos": "VERB",
    "xpos": "VBN",
    "feats": "Tense=Past|VerbForm=Part|Voice=Pass",
    "head": 0,
    "deprel": "root",
    "start_char": 41,
    "end_char": 48
  }
] [] ------------

[
  {
    "id": 4,
    "text": "president",
    "lemma": "president",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 3,
    "deprel": "xcomp",
    "start_char": 49,
    "

## FEATURE EXTRACTION PART

In [23]:
def lookParent(sent,tok,d,pospath):
  print(tok.text,d)
  if tok.head==0:
    pospath.append(tok.pos)
    return d,pospath
  else:
    
    d+=1
    pospath.append(tok.pos)
    headword=sent.words[tok.head-1]
    return lookParent(sent,headword,d,pospath)


In [24]:
from prompt_toolkit.widgets.dialogs import D
headsID_list=[]
headsword_list=[]
deprel_list=[]
pathlen=[]
pospath_list=[]
sent=en_doc.sentences[1]

for tok in sent.words:
  print("\nCONSIDERING ",tok.text) #EXTRACT WHATS NECESSATY for TOKEN AND PUT IT IN A TOUPLE

  #GET HEAD ID FEATURE
  head=tok.head
  print("headID",head)
  headsID_list.append(tok.head)


  #GET HEAD WORD
  if word.head > 0:
    head_word=sent.words[tok.head-1].text
  else:
    head_word= "root"
  print("head word",head_word)
  headsword_list.append(head_word)

  #GET DEPEPDENCY RELATION WITH ITS HEAD
  deprel=word.deprel
  deprel_list.append(deprel)
  print("deprel: ",deprel)


  print(" number of arcs to root + POStagspath to root:\n")
  pospath=[]
  path ,postagspath=lookParent(sent,tok,0,pospath)

  print(path,postagspath)
  pathlen.append(path)
  pospath_list.append(postagspath)


He
headID 3
head word elected
deprel:  punct
 path to root:

He 0
elected 1
1 ['PRON', 'VERB']
-----------
was
headID 3
head word elected
deprel:  punct
 path to root:

was 0
elected 1
1 ['AUX', 'VERB']
-----------
elected
headID 0
head word .
deprel:  punct
 path to root:

elected 0
0 ['VERB']
-----------
president
headID 3
head word elected
deprel:  punct
 path to root:

president 0
elected 1
1 ['NOUN', 'VERB']
-----------
in
headID 6
head word 2008
deprel:  punct
 path to root:

in 0
2008 1
elected 2
2 ['ADP', 'NUM', 'VERB']
-----------
2008
headID 3
head word elected
deprel:  punct
 path to root:

2008 0
elected 1
1 ['NUM', 'VERB']
-----------
.
headID 3
head word elected
deprel:  punct
 path to root:

. 0
elected 1
1 ['PUNCT', 'VERB']
-----------


In [ ]:
   # Create the DataFrame
  df = pd.DataFrame({
"headsID_list":headsID_list,
"headsword_list":headsword_list,
"deprel_list":deprel_list,
"pathlen":pathlen,
"pospath_list":pospath_list      
    })